In [ ]:
channel_lines = calculate_channel_flow_lines(vertices, triangle_objects)
print(len(channel_lines))
print(channel_lines)

In [ ]:
max_length = 0
for line in channel_lines:
    if len(line) > max_length:
        max_length = len(line)
max_length

In [ ]:
# get vertex closest to coord
coord = (-116.56625, 49.50462)
closest_vertex = None
min_dist = float('inf')
for vertex in vertices.values():
    dist = np.sqrt((vertex.x - coord[0])**2 + (vertex.y - coord[1])**2)
    if dist < min_dist:
        min_dist = dist
        closest_vertex = vertex

print(closest_vertex.x, closest_vertex.y, closest_vertex.z)

In [ ]:
# draw triangles
fig = plt.figure()
ax = fig.add_subplot(111)
for triangle in triangles_subset:
    draw_triangle(ax, triangle, "#00000055", xs, ys)

print("Finished drawing triangles.")

# draw lines
for line in channel_lines:
    thickness = 1
    thickness_growth = 1
    color = (0, 0, 1)
    color_growth = (0.05, 0, 0)

    prev_point = line[0]
    #draw_vertex(ax, prev_point, markersize=5)
    for point in line[1:]:
        draw_line(ax, [prev_point, point], "blue", linewidth=thickness)
        color = (min(color[0] + color_growth[0], 1), min(color[1] + color_growth[1], 1), min(color[2] + color_growth[2], 1))
        thickness += thickness_growth
        prev_point = point

draw_vertex(ax, closest_vertex, markersize=8)

"""
verts = get_vertices_on_area_bounds(vertices)
verts.extend(get_vertices_in_pit(vertices))
for vert in verts:
    draw_vertex(ax, vert, markersize=5)

    ordered = get_all_triangles_and_edges_at_point(vert, triangle_objects)

    previous_item = None
    next_item = None
    j = 0
    next_point = None
    for item in ordered:
        previous_item = ordered[(j - 1) % len(ordered)]
        next_item = ordered[(j + 1) % len(ordered)]

        if isinstance(item, Triangle):
            pass # do nothing for triangles
        else: # must be an edge
            draw_line(ax, [item[0], item[1]], "green", linewidth=2)

            if item[0].id != vert.id:
                item = (item[1], item[0]) # make first vertex be the given vertex
            
            #lies_in_channel = edge_lies_in_channel(item[0], item[1], previous_item, next_item)
            lies_in_channel = False
            ascending = item[0].z <= item[1].z

            # next_item and previous_item should be triangles adjacent to this edge
            if ascending and lies_in_channel:
                next_point = get_other_vertex_from_edge(vert, item)
                next_lines = channel_flow_line_helper(next_point, triangles) # keep going up the channel
                new_lines = [[item[1]]]
                for line in next_lines:
                    new_line = [item[1]]
                    new_line.extend(line)
                    new_lines.append(new_line)
                
                lines.extend(new_lines)"""

print("Done drawing lines.")

plt.show()

In [ ]:
p = pv.Plotter()
p.add_mesh(get_mesh_from_triangles(triangles_subset, vertices_3d), show_edges=False, color='lightgray')

z_offset = 0.0001
previous_percentange = 0

DRAW_CHANNEL_LINES = False

if DRAW_CHANNEL_LINES:
    i = 0
    for line in channel_lines:
        prev_point = line[0]

        new_line = []
        p2 = None
        for point in line[1:]:
            p1 = [prev_point.x, prev_point.y, (prev_point.z * z_scale) + z_offset]
            p2 = [point.x, point.y, (point.z * z_scale) + z_offset]
            new_line.append(p1)
            new_line.append(p2)
            prev_point = point

        p.add_lines(np.array(new_line), color='blue', width=5)

        i += 1
        current_percentage = int((i / len(channel_lines)) * 100)
        if current_percentage != previous_percentange:
            print(f"Progress: {current_percentage}%")
            previous_percentange = current_percentage

    print("Done drawing lines.")

#p.show(jupyter_backend='static')
#p.show(jupyter_backend='html') # simple interactive
p.show() # fully interactive